# フードバンクにおける食品分配問題
深層強化学習の適用

DDQN (Double Deep Q-Network)

ニューラルネットワークにはPyTorchを使用

In [ ]:
# パッケージのimport
import numpy as np
import matplotlib.pyplot as plt
from typing import List
import copy
import datetime
import os
%matplotlib inline

In [ ]:
# namedtupleを生成
from collections import namedtuple

Transition = namedtuple(
    'Transition', ('state', 'action', 'next_state', 'reward'))

### 状態定義

In [ ]:
# 状態定義
from enum import IntEnum

# 在庫の残り
class StockRemaining(IntEnum):
    FULL = 0
    MANY = 1
    FEW = 2
    NONE = 3

# 在庫の変動
class StockChange(IntEnum):
    NONE = 0
    SLIGHTLY = 1
    SOMEWHAT = 2
    GREATLY = 3

# エージェントの充足度
class Satisfaction(IntEnum):
    HARDLY = 0
    SOMEWHAT = 1
    COMLETELY = 2
    OVERLY = 3

# 終端状態の区別用
class Progress(IntEnum):
    ONGOING = 0
    DONE = 1

### ハイパーパラメータ

In [ ]:
# 学習用パラメータ
NUM_EPISODES = 10001
MAX_STEPS = 100
GREEDY_CYCLE = 100

GAMMA = 0.98

INITIAL_EPSILON = 1.0
MINIMUM_EPSILON = 0.6
EPSILON_DELTA = (INITIAL_EPSILON - MINIMUM_EPSILON) / (NUM_EPISODES * 0.95)

ALPHA = 0.0001
# INITIAL_ALPHA = 0.1
# MINIMUM_ALPHA = 0.0001
# ALPHA_DELTA = (INITIAL_ALPHA - MINIMUM_ALPHA) / (NUM_EPISODES * 0.95)

HIDDEN_SIZE = 32

BATCH_SIZE = 32
CAPACITY = 10000

TARGET_UPDATE = 10



# 環境設定

# AGENTS_COUNT = 1
# FOODS = [5, 5, 5]
# NUM_FOODS = len(FOODS)
# REQUESTS = [
#     [1, 3, 5],
# ]

AGENTS_COUNT = 3
FOODS = [20, 20, 20]
NUM_FOODS = len(FOODS)
REQUESTS = [
    [10, 10, 10],
    [5, 10, 5],
    [5, 5, 10],
]


### 経験再生

In [ ]:
# 経験を保存するメモリクラスを定義

from collections import deque

class ReplayMemory:
    """
    Experience Replay(経験再生)を実装するクラス
    学習データを一定数メモリに保存し、ランダムにサンプリングすることで
    時系列の相関をなくすことができる
    """

    def __init__(self, capacity):
        """メモリを初期化"""
        # 状態遷移を保持するバッファ
        self.memory = deque([], maxlen=capacity)
        # deque: double-ended queue （両端キュー）
        # 両サイドからデータを取り出したり追加できる
        # リストに比べて先頭・末尾の操作がO(1)の計算量で済む
        # 最大要素数のcapacityを超えると古い要素から削除される

    def push(self, state, action, state_next, reward):
        '''transition = (state, action, state_next, reward)をメモリに保存する'''
        
        tensor_state = torch.tensor(state, dtype=torch.float).unsqueeze(0)
        tensor_state_next = torch.tensor(state_next, dtype=torch.float).unsqueeze(0)
        tensor_action = torch.tensor([action], dtype=torch.long).unsqueeze(0)
        tensor_reward = torch.tensor([reward], dtype=torch.long)

        # namedtupleのTransitionを使用し、値とフィールド名をペアにして保存します
        t = Transition(tensor_state, tensor_action, tensor_state_next, tensor_reward)
        self.memory.append(t)


    def sample(self, batch_size):
        '''batch_size分だけ、ランダムに保存内容を取り出す'''
        return random.sample(self.memory, batch_size)

    def __len__(self):
        '''関数lenに対して、現在の変数memoryの長さを返す'''
        return len(self.memory)


### Deep Neural Network

In [ ]:
# ディープ・ニューラルネットワークの構築
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self, n_in, n_mid, n_out):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_in, n_mid)
        self.fc2 = nn.Linear(n_mid, n_mid)
        self.fc3 = nn.Linear(n_mid, n_out)

    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        output = self.fc3(h2)
        return output

### Double DQN

In [ ]:
# エージェントが持つ脳

import random
import torch
from torch import nn
from torch import optim

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.kaiming_normal_(m.weight)
        nn.init.zeros_(m.bias)

class Brain:
    def __init__(self, num_states, num_actions, f):
        self.f = f
        self.num_actions = num_actions

        # 経験を記憶するメモリオブジェクトを生成
        self.memory = ReplayMemory(CAPACITY)

        # ニューラルネットワークを構築
        n_in, n_mid, n_out = num_states, 32, num_actions
        self.main_q_network = Net(n_in, n_mid, n_out)  # Netクラスを使用
        self.target_q_network = Net(n_in, n_mid, n_out)  # Netクラスを使用
        print(self.main_q_network)  # ネットワークの形を出力

        # 最適化手法の設定
        self.optimizer = optim.Adam(self.main_q_network.parameters(), lr=ALPHA)

        # 損失関数
        self.criterion = nn.SmoothL1Loss()

        # self.model.apply(init_weights)
    
    def replay(self, greedy):
        '''Experience Replayでネットワークの結合パラメータを学習'''

        # メモリに十分にたまるまで待つ
        if len(self.memory) < BATCH_SIZE:
            return

        # ミニバッチの作成
        self.batch, self.state_batch, self.action_batch, self.reward_batch, self.non_final_next_states = self.make_minibatch()

        # 教師信号となるQ(s_t, a_t)値を求める
        self.target_q_values = self.get_target_q_values()

        # 結合パラメータの更新
        self.update_main_q_network()


    def decide_action(self, state, options, greedy, epsilon):
        '''現在の状態に応じて、行動を決定する'''
        # ε-greedy法で徐々に最適行動のみを採用する
        # epsilon = 0.5 * (1 / (episode + 1))
        greedy_print = greedy


        if greedy is False:
            # ε-greedyで行動を決定する
            if np.random.rand() >= epsilon:
                greedy = True


        if greedy:
            self.main_q_network.eval()  # ネットワークを推論モードに切り替える

            tensor_state = torch.tensor(state, dtype=torch.float)

            # print(state)
            # print(tensor_state)
        
            with torch.no_grad():
                q = self.main_q_network(tensor_state).numpy()
                # print(f"Q値: {q}")
                options_q = q[options]
                max_arg = np.argmax(options_q)
                action = options[max_arg]
                # print(f"action: {action}")
                if greedy_print:
                    print(f"Q値: {q} 選択した行動: {action}", file=f)
                
            # ネットワークの出力の最大値のindexを取り出す

        else:
            action = random.choice(options)
            # print(f"ランダムな行動: {action}")

        return action

    def make_minibatch(self):
        '''ミニバッチの作成'''

        # メモリからミニバッチ分のデータを取り出す
        transitions = self.memory.sample(BATCH_SIZE)

        # 各変数をミニバッチに対応する形に変形
        batch = Transition(*zip(*transitions))

        # 各変数の要素をミニバッチに対応する形に変形し、ネットワークで扱えるようVariableにする
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)

        # 次状態のうち、終端状態でない部分を抜き出す
        non_final_next_states = torch.cat([s for s in batch.next_state if s.numpy()[0][-1] != Progress.DONE])

        return batch, state_batch, action_batch, reward_batch, non_final_next_states

    def get_target_q_values(self):
        '''教師信号となるQ(s_t, a_t)値を求める'''

        # ネットワークを推論モードに切り替える
        self.main_q_network.eval()
        self.target_q_network.eval()

        # 現在のQ値（メインネットワークの出力）
        self.current_q_values = torch.gather(self.main_q_network(self.state_batch), 1, self.action_batch)


        # 終端状態でない次状態のマスク（終端でない所がTrueになる）
        non_final_mask = torch.tensor(tuple(map(lambda s: s.numpy()[0][-1] != Progress.DONE, self.batch.next_state)))

        # print(self.non_final_next_states)

        non_final_q_values  = self.target_q_network(self.non_final_next_states).detach()
        non_final_possible_q_values =  torch.tensor([ [non_final_q_values[si, a] if s[a] != StockRemaining.NONE else float("-inf") for a in range(NUM_FOODS + 1)  ] for si, s in enumerate(self.non_final_next_states.detach())])


        
        next_max_q_values = torch.zeros(BATCH_SIZE) # すべて0にしておく
        # 次状態のQ値の最大値（ターゲットネットワークの出力）
        # 終端状態でないインデックスに、次状態の最大Q値を挿入
        next_max_q_values[non_final_mask] = non_final_possible_q_values.max(1)[0].detach()

        

        # if greedy:
            # print(non_final_q_values)
            # print(possible_action_index)
            # print(self.main_q_network(self.state_batch))
            # test_batch  = torch.tensor([[0, 1]] * len(self.non_final_next_states))
            # print(test_batch)
            # print(self.current_q_values)
            
            # print(non_final_q_values)
            # print(self.non_final_next_states)
            # print(non_final_possible_q_values)
            # print(test_batch)
            # print(torch.tensor(possible_action_index))
            # print(torch.gather(non_final_q_values, 1, test_batch))
        

        # next_Q = []
        # for i in range(es.NUM_FOODS):
        #     if state_next[i] != StockRemaining.NONE:
        #         next_Q.append(self.Q[state_next][i])
        # next_Q.append(self.Q[state_next][-1])

        # max_Q = max(next_Q)

        # print(next_max_q_values)


        # 教師データとなるQ値の目標値
        # （報酬 + 割引率 * 次状態の最大Q値）
        target_q_values = (self.reward_batch + GAMMA * next_max_q_values).unsqueeze(1)

        return target_q_values
    
    def update_main_q_network(self):
        '''4. 結合パラメータの更新'''

        # ネットワークを訓練モードに切り替える
        self.main_q_network.train()

        # 損失関数を計算する（smooth_l1_lossはHuberloss）
        # expected_state_action_valuesは
        # sizeが[minbatch]になっているので、unsqueezeで[minibatch x 1]へ
        loss = self.criterion(self.current_q_values,
                                self.target_q_values)

        # 結合パラメータを更新する
        self.optimizer.zero_grad()  # 勾配をリセット
        loss.backward()  # バックプロパゲーションを計算

        nn.utils.clip_grad_norm_(self.main_q_network.parameters(), 1.0)
        self.optimizer.step()  # 結合パラメータを更新

    def update_target_q_network(self):  # DDQNで追加
        '''Target Q-NetworkをMainと同じにする'''
        self.target_q_network.load_state_dict(self.main_q_network.state_dict())
        # print("Target Q-Networkを更新")


### エージェント

In [ ]:
# Agentクラス

class Agent:
    def __init__(self, name, request, num_states, num_actions,  f):
        self.name = name
        self.REQUEST = request
        self.brain = Brain(num_states, num_actions, f)
        self.f = f

    def reset(self, env_stock, greedy):
        self.current_requests = self.REQUEST.copy()
        self.stock = np.zeros(NUM_FOODS, dtype=np.int64)

        self.food_complete = False
        self.learning_done = False
        self.old_env_stock = env_stock.copy()



    def learn(self, greedy):
        self.brain.replay(greedy)

    def update_target_q_function(self):
        '''Target Q-NetworkをMain Q-Networkと同じに更新'''
        
        self.brain.update_target_q_network()        


    # 行動（どの食品を取得するか）を決定
    def get_action(self, state, env_stock, greedy, epsilon):
        # 行動の候補
        action_options = []

        # 本部に在庫がある食品を候補に入れる
        for food in range(len(env_stock)):
            if env_stock[food] != 0:
                action_options.append(food)


        # 「何もしない」という選択肢も候補に加える
        action_options.append(NUM_FOODS)


        # 行動を決定
        action = self.brain.decide_action(state, action_options, greedy, epsilon)

        return action


    def memorize(self, state, action, state_next, reward, greedy):
        '''memoryオブジェクトに、state, action, state_next, rewardの内容を保存する'''
        if state == state_next:
            if greedy:
                print(f"{self.name}: 状態が変化していません", file=self.f)
        else:
            # if greedy:
            #     print("--- メモリに保存 ---", file=self.f)
            #     print("現在の状態: ", file=self.f)
            #     self.print_state(state)
            #     print(f"とった行動: {action}", file=self.f)
            #     print("次の状態: ", file=self.f)
            #     self.print_state(state_next)
            #     print(f"獲得した報酬: {reward}", file=self.f)
            #     print("-----------------", file=self.f)
            if not greedy:
                self.brain.memory.push(state, action, state_next, reward)


    def observe_state(self, env_stock, episode_terminal):
        remainings = []
        changes = []
        satisfactions = []
        progress = []

        granularity = len(StockRemaining) - 2
        for amount, original in zip(env_stock, FOODS):
            section = round(original / granularity)
            if amount == 0:
                remainings.append(StockRemaining.NONE)
            elif amount < section:
                remainings.append(StockRemaining.FEW)
            elif amount < original:
                remainings.append(StockRemaining.MANY)
            else:
                remainings.append(StockRemaining.FULL)

        difference = self.old_env_stock - env_stock
        for diff in difference:
            if diff == 0:
                changes.append(StockChange.NONE)
            elif diff == 1:
                changes.append(StockChange.SLIGHTLY)
            elif diff == 2:
                changes.append(StockChange.SOMEWHAT)
            else:
                changes.append(StockChange.GREATLY)

        satisfaction_rates = self.stock / self.REQUEST
        for rate in satisfaction_rates:
            if rate < 0.5:
                satisfactions.append(Satisfaction.HARDLY)
            elif rate < 1:
                satisfactions.append(Satisfaction.SOMEWHAT)
            elif rate == 1:
                satisfactions.append(Satisfaction.COMLETELY)
            else:
                satisfactions.append(Satisfaction.OVERLY)

        if episode_terminal:
            progress.append(Progress.DONE)
        else:
            progress.append(Progress.ONGOING)

        state = tuple(remainings + changes + satisfactions + progress)

        # state = tuple(remainings + satisfactions + progress)

        self.old_env_stock = env_stock.copy()

        return state


    def grab_food(self, food):
        # 手元の在庫が1つ増える
        self.stock[food] += 1
        # 要求リストから1つ減らす
        if (self.current_requests[food] != 0):
            self.current_requests[food] -= 1

        # self.check_food_complete()

    def check_food_complete(self, env_stock):
        # print(f"要求: {self.current_requests}")
        if np.all(self.current_requests == 0):
            # 要求がすべて満たされたことを記録
            return True
            # print(f"{self.name} 要求がすべて満たされました")
        else:
            # 要求が満たされてない場合、食品がまだ余っているか在庫をチェック
            # 欲しい食品の在庫
            required_food_stock =  env_stock[self.current_requests > 0]
            # 欲しい食品の在庫がすべてない
            if np.all(required_food_stock == 0):
                return True
        return False


    def get_violation(self):
        diffs = self.REQUEST - self.stock
        diff_rates = diffs / self.REQUEST * 10
        abs_diffs = np.absolute(diff_rates)
        violation = np.sum(abs_diffs)

        # if (abs(violation) > 100):
        #     print(f"violation: {violation}")

        self.violation = violation
        return violation


    def print_state(self, state):
        num = NUM_FOODS
        print(f"{self.name} State: ", end="", file=self.f)

        print("Remaining[ ", end="", file=self.f)
        for i in range(num):
            print(f"{state[i].name} ", end="", file=self.f)
        print("], Change[ ", end="", file=self.f)
        for i in range(num, num * 2):
            print(f"{state[i].name} ", end="", file=self.f)
        print("], Satisfaction[ ", end="", file=self.f)
        for i in range(num * 2, num * 3):
            print(f"{state[i].name} ", end="", file=self.f)

        print(f"] Progress[{state[num * 3].name}]", file=self.f)


### 環境

In [ ]:
class Environment:

    def __init__(self, f):
        self.f = f
        self.agents = self.init_agents()

    def init_agents(self):
        state_size = pow(len(StockRemaining), NUM_FOODS) * pow(len(StockChange),
                                                                  NUM_FOODS) * pow(len(Satisfaction), NUM_FOODS) * len(Progress)
        num_states = NUM_FOODS * 3 + 1
        num_actions = NUM_FOODS + 1

        print("========= 各エージェントのもつ状態行動空間 =========", file=self.f)
        print(f"状態数: {state_size:,}", file=self.f)
        print(f"行動数: {num_actions}", file=self.f)
        print(f"状態 × 行動の組み合わせ: {(state_size * num_actions):,}", file=self.f)
        print("\n\n", file=self.f)

        agents: List[Agent] = []
        for i in range(AGENTS_COUNT):
            name = f"Agent{i + 1}"
            agent = Agent(
                name, np.array(REQUESTS[i]), num_states, num_actions, self.f)
            agents.append(agent)
        return agents

    def reset(self, greedy):
        self.stock = np.array(FOODS, dtype=np.int64)
        states = None

        for agent in self.agents:
            agent.reset(self.stock, greedy)

        return states

    def get_actions(self, states):
        actions = []
        # すべてのエージェントに対して
        for agent, state in zip(self.agents, states):
            # 行動を決定
            action = agent.get_action(state)
            actions.append(action)
            if action == NUM_FOODS:
                # print(f"{agent.name} 行動: 何もしない")
                pass
            else:
                # print(f"{agent.name} 行動: 食品{action}を１つ取る")
                pass
        return actions

    def check_food_run_out(self):
        # 全ての在庫が0になったかチェック
        return np.all(self.stock == 0)

    def check_agents_food_complete(self):
        # 全てのエージェントが終了条件を満たしているかチェック
        all_done = True
        for agent in self.agents:
            if not agent.check_food_complete(self.stock):
                all_done = False
                break
        # 全エージェントの取れる行動がなくなったか
        return all_done

    def check_agents_learning_done(self):
        all_done = True
        for agent in self.agents:
            if not agent.learning_done:
                all_done = False
                break
        return all_done

    # def learn(self, states, actions, rewards, states_next, alpha):
    #     for agent, state, action, reward, state_next in zip(self.agents, states, actions, rewards, states_next):
    #         agent.learn(state, action, reward, state_next, alpha)

    def get_reward(self, target_agent: Agent, terminal, greedy):
        if terminal:
            # - |制約違反度の平均からの偏差|
            # violations = []
            # for agent in self.agents:
            #     v = agent.get_violation()
            #     violations.append(v)
            # mean = np.mean(violations)
            # abs_deviation = np.absolute(mean - target_agent.get_violation())
            # reward = - abs_deviation

            # - (制約違反度 + 制約違反度の標準偏差)
            violations = []
            for agent in self.agents:
                v = agent.get_violation()
                # print(f"violation: {v}")
                violations.append(v)
            std = np.std(violations)
            # print(f"std: {std}")
            
            reward = - (target_agent.get_violation() + std * 10)
            # reward = 0
            # print(f"reward: {reward}")

            # - (制約違反度の平均+標準偏差)　統一
            # violations = []
            # for agent in self.agents:
            #     v = agent.get_violation()
            #     violations.append(v)
            # mean = np.mean(violations)
            # std = np.std(violations)
            # reward = - (mean + std)

            if greedy:
                print(
                    f"{target_agent.name}: 報酬{reward:.3f}  要求{target_agent.REQUEST} 在庫{target_agent.stock}", file=self.f)

        else:
            reward = -1
            # reward = 0
       
        return reward

    def print_env_state(self):
        print("Env State: [", end="", file=self.f)
        for status in self.env_state:
            print(f"{status.name} ", end="", file=self.f)

        print("]", file=self.f)

### メイン

In [ ]:


np.set_printoptions(precision=5, floatmode='maxprec_equal')
np.set_printoptions(suppress=True)

DIR_PATH = "D:\\Lighthouse\\Documents\\Reinforcement Learning\\Food Distribution\\results"

start_time = datetime.datetime.now()
file_name_time = "{0:%Y-%m-%d_%H%M%S}".format(start_time)

log_name = f"log_{file_name_time}.txt"
log_path = os.path.join(DIR_PATH, "logs", log_name)
f = open(log_path, mode="w", encoding="UTF-8")

print("フードバンク食品分配 DDQN\n", file=f)

print("開始時刻: {0:%Y/%m/%d %H:%M:%S}\n".format(start_time), file=f)

print("====================== 環境設定 ======================", file=f)
print(f"食品の種類: {NUM_FOODS}", file=f)
print(f"本部の在庫: {FOODS}", file=f)
print(f"エージェント数: {AGENTS_COUNT}", file=f)
print(f"エージェントの要求リスト: {REQUESTS}", file=f)
print("\n\n", file=f)

print("==================== 学習パラメーター ====================", file=f)
print(f"エピソード数: {NUM_EPISODES:,}", file=f)
print(f"最大ステップ数: {MAX_STEPS:,}\n", file=f)
print(f"割引率: {GAMMA}\n", file=f)
print(f"学習率: {ALPHA}", file=f)
print(f"εの初期値: {INITIAL_EPSILON}", file=f)
print(f"εの最終値: {MINIMUM_EPSILON}", file=f)
print(f"1エピソードごとのεの減少値: {EPSILON_DELTA:.7f}\n", file=f)
print(f"隠れニューロン数: {HIDDEN_SIZE}", file=f)
print(f"経験再生メモリ容量: {CAPACITY}", file=f)
print(f"バッチサイズ: {BATCH_SIZE}", file=f)
print(f"ターゲットネットワークの更新間隔EP: {TARGET_UPDATE}", file=f)
print("\n\n", file=f)

print("====================== 報酬設定 ======================", file=f)
print("- (制約違反度 + 制約違反度の標準偏差)", file=f)
# print("- |制約違反度の平均からの偏差|", file=f)
# print("- (制約違反度の平均 + 標準偏差) 統一", file=f)
print("\n\n", file=f)

env = Environment(f)

result_episodes = []
results_agents_y = [[], [], []]

result_ave = []
result_dev = []

epsilon = INITIAL_EPSILON

fig = plt.figure()

ax1 = fig.add_subplot(3, 1, 1)
ax2 = fig.add_subplot(3, 1, 2)
ax3 = fig.add_subplot(3, 1, 3)

ax1.set_xlabel("Episode")
ax2.set_xlabel("Episode")
ax3.set_xlabel("Episode")

ax1.set_ylabel("Constraint Violations")
ax2.set_ylabel("Mean")
ax3.set_ylabel("Variance")

line1, = ax1.plot([], [], label="Agent1")
line2, = ax1.plot([], [], label="Agent2")
line3, = ax1.plot([], [], label="Agent3")

line_ave, = ax2.plot([], [])
line_dev, = ax3.plot([], [])

ax1.legend()

lines = [line1, line2, line3]

# 各エピソード
for episode in range(NUM_EPISODES):

    if episode % GREEDY_CYCLE == 0:
        greedy = True
        print(
            f"-------------- Episode:{episode} (greedy) --------------")
        print(
            f"\n\n-------------- Episode:{episode} (greedy) ----------------------------------------------------------------------", file=f)
    else:
        greedy = False
        # print(f"-------------- Episode:{episode} --------------")
    states = [[], [], []]
    actions = [[], [], []]
    rewards = [0, 0, 0]

    old_states = env.reset(greedy)
    old_actions = []

    learning_complete = False
    step = 0

    if greedy:
        # print(f"学習率: {alpha:.5f}", file=f)
        print(f"ε: {epsilon:.5f}", file=f)

    # 各ステップ
    while True:
        if greedy:
            print(f"\n------- Step:{step} -------", file=f)

        # 各エージェント
        for index, agent in enumerate(env.agents):

            # 終了条件確認
            # food_run_out = env.check_food_run_out()
            all_agents_food_compolete = env.check_agents_food_complete()
            exceed_max_step = step == MAX_STEPS - 1

            episode_terminal = all_agents_food_compolete or exceed_max_step

            if greedy:
                print(f"\n本部の在庫: {env.stock}", file=f)
                print(f"{agent.name}の在庫:{agent.stock} 要求:{agent.current_requests}", file=f)
                diff = agent.old_env_stock - env.stock
                print(f"{agent.name}視点の本部在庫の変化:{diff}", file=f)
                if episode_terminal:
                    print("*** 終了条件を満たしています ***", file=f)

            # 状態を観測
            state = agent.observe_state(env.stock, episode_terminal)
            states[index] = state

            if greedy:
                agent.print_state(state)

            # 行動を決定
            if not episode_terminal:
                action = agent.get_action(
                    state, env.stock, greedy, epsilon)
                actions[index] = action

                if greedy:
                    if action == NUM_FOODS:
                        action_string = "何もしない"
                    else:
                        action_string = f"食品{action}"

                    print(
                        f"{agent.name} 選択した行動: {action_string}", file=f)

                # 行動をとる
                if action != NUM_FOODS:
                    # エージェントが食品を1つとる
                    agent.grab_food(action)
                    # 本部の在庫が1つ減る
                    env.stock[action] -= 1

            reward = env.get_reward(agent, episode_terminal, greedy)

            if old_states is not None:
                agent.memorize(old_states[index], old_actions[index], state, reward, greedy)
                agent.learn(greedy)

            if episode_terminal:
                if greedy:
                    print(f"{agent.name} 学習終了\n", file=f)

                rewards[index] = reward
                agent.learning_done = True
                learning_complete = env.check_agents_learning_done()
                if learning_complete:
                    break

        if exceed_max_step and greedy:
            print("最大ステップ数を超えました", file=f)

        if learning_complete:
            break

        old_states = copy.deepcopy(states)
        old_actions = copy.deepcopy(actions)

        step += 1


    if epsilon > MINIMUM_EPSILON:
        epsilon = epsilon - EPSILON_DELTA

    if (episode % TARGET_UPDATE == 0):
        for agent in env.agents:
            agent.update_target_q_function()

    if greedy:
        print(
            f"************ Episode{episode}の結果 ************", file=f)
        print(f"要したステップ数: {step + 1}", file=f)
        print(f"要したステップ数: {step + 1}")
        print(f"食品の残り個数: {np.sum(env.stock)}", file=f)
        violations = []
        for agent, reward in zip(env.agents, rewards):
            print(
                f"{agent.name}の在庫: {agent.stock}  制約違反度: {agent.violation:.3f}  報酬: {reward:.3f}", file=f)
            print(f"{agent.name} reward: {reward:.3f}")
            violations.append(agent.violation)

        mean = np.average(violations)
        var = np.std(violations)

        print(f"制約違反度の平均: {mean:.3f}  分散: {var:.3f}", file=f)
        # print(f"現在のエピソード: {episode}")
        # result_reward_x.append(episode)
        # result_reward_y.append(rewards[0])
        # result_optimal_reward_x.append(episode)
        # result_optimal_reward_y.append(optimal_reward)
        # lines1.set_data(result_reward_x, result_reward_y)
        result_episodes.append(episode)
        result_ave.append(mean)
        result_dev.append(var)

        for i, agent, line, result in zip(range(AGENTS_COUNT), env.agents, lines, results_agents_y):
            # result.append(rewards[i])
            result.append(violations[i])
            line.set_data(result_episodes, result)

        line_ave.set_data(result_episodes, result_ave)
        line_dev.set_data(result_episodes, result_dev)

        # lines1.set_data(result_agents_x, result_agent1_y)
        # lines2.set_data(result_agents_x, result_agent2_y)
        # lines3.set_data(result_agents_x, result_agent3_y)

        ax1.relim()
        ax1.autoscale_view()

        ax2.relim()
        ax2.autoscale_view()

        ax3.relim()
        ax3.autoscale_view()

        # lines1.set_data(result_satisfaction_x, result_satisfaction_y)

        # lines2.set_data(result_optimal_reward_x, result_optimal_reward_y)
        plt.pause(0.001)

# print("\n---- 最適解 ----")
# for agent, loss, stock, request in zip(env.agents, optimal_loss_values, optimal_agent_stock, REQUESTS):
#     diff = stock - request
#     print(f"{agent.name}: 損失{loss:.1f} 要求との差{diff} 在庫{stock}")
# print(f"報酬: {optimal_reward:.4f}")
# print(f"発見したエピソード: {optimal_episode}")

end_time = datetime.datetime.now()
print("\n終了時刻: {0:%Y/%m/%d %H:%M:%S}".format(end_time), file=f)

figure_name = f"figure_{file_name_time}.png"
figure_path = os.path.join(DIR_PATH, "figures", figure_name)
# plt.savefig(figure_path)

# csv_name = f"data_{file_name_time}.csv"
# csv_path = os.path.join(DIR_PATH, "data", csv_name)
# with open(csv_path, "w", newline='', encoding="UTF-8") as csv_f:
#     writer = csv.writer(csv_f)
#     writer.writerow(["Episode", "Distance"])
#     writer.writerows(self.result)

print("\n\n終了")

plt.show()

f.close()